In [0]:
!git clone https://github.com/muellerzr/fastai-Experiments-and-tips.git
%cd fastai-Experiments-and-tips/
from Tabular_MixUp import *

# Tabular MixUp

In [0]:
from fastai.tabular import *

In [0]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')

In [0]:
dep_var = 'salary'
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [FillMissing, Categorify, Normalize]

In [32]:
len(df)

32561

In [0]:
train = df.iloc[0:int(len(df) * .7)]
test = df.iloc[int(len(df) * .7):]

In [6]:
len(train), len(test)

(22792, 9769)

# Ten Times

Each will be run with early stopping

In [0]:
from fastai.callbacks import EarlyStoppingCallback

## Base

In [25]:
dep_var

'salary'

In [0]:
def calcError(arr:np.array):
  mean = arr.mean()
  stds = []
  for _, val in enumerate(arr):
    stds.append((mean-val)**2)
  stds = np.array(stds).sum()
  var = math.sqrt(stds/(len(arr)-1))
  return var/math.sqrt(len(arr))

In [0]:
def calc_mean(l): return reduce(lambda x, y: x + y, l) / len(l)
mostconf = []
valid_avg = []
test_avg = []
callbacks = [TabMixUpCallback, EarlyStoppingCallback]

procs = [Categorify, Normalize, FillMissing]
loss_func=CrossEntropyFlat()

for x in range(10):
  dep_var = 'salary'
  cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
  cont_names = ['age', 'fnlwgt', 'education-num']
  procs = [FillMissing, Categorify, Normalize]
  data = (TabularList.from_df(train, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_rand_pct()
                           .label_from_df(cols=dep_var)
                           .databunch())
  learn = mytabular_learner(data, layers=[200,100], metrics=accuracy, callback_fns = callbacks)
  learn.fit_one_cycle(5, max_lr=1e-3)

  valid_avg.append(float(learn.validate()[1]))

  data_test = (TabularList.from_df(test, path = '', cat_names = cat_names, cont_names = cont_names, procs = procs, processor=data.processor)
       .split_none()
       .label_from_df(cols=dep_var))

  data_test.valid = data_test.train
  data_test = data_test.databunch()
  learn.data.valid_dl = data_test.valid_dl
  test_avg.append(float(learn.validate()[1]))

  interp = ClassificationInterpretation.from_learner(learn)
  mostconf.append(interp.confusion_matrix())

  print(f'Validation Accuracy {x+1}: {valid_avg[x]}')
  print(f'Test Accuracy {x+1}: {test_avg[x]}')


In [44]:
print(f'Average Validation Accuracy: {calc_mean(valid_avg)}')
print(f'Validation Error: {calcError(np.array(valid_avg))}')
print(f'Average Test Accuracy: {calc_mean(test_avg)}')
print(f'Test Error: {calcError(np.array(test_avg))}')

Average Validation Accuracy: 0.8406318545341491
Validation Error: 0.0013322007412085754
Average Test Accuracy: 0.8356945395469666
Test Error: 0.0005333146311434077


In [46]:
mostconf = []
valid_avg = []
test_avg = []
callbacks = [EarlyStoppingCallback]

procs = [Categorify, Normalize, FillMissing]
loss_func=CrossEntropyFlat()

for x in range(10):
  dep_var = 'salary'
  cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
  cont_names = ['age', 'fnlwgt', 'education-num']
  procs = [FillMissing, Categorify, Normalize]
  data = (TabularList.from_df(train, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_rand_pct()
                           .label_from_df(cols=dep_var)
                           .databunch())
  learn = tabular_learner(data, layers=[200,100], metrics=accuracy, callback_fns = callbacks)
  learn.fit_one_cycle(5, max_lr=1e-3)

  valid_avg.append(float(learn.validate()[1]))

  data_test = (TabularList.from_df(test, path = '', cat_names = cat_names, cont_names = cont_names, procs = procs, processor=data.processor)
       .split_none()
       .label_from_df(cols=dep_var))

  data_test.valid = data_test.train
  data_test = data_test.databunch()
  learn.data.valid_dl = data_test.valid_dl
  test_avg.append(float(learn.validate()[1]))

  interp = ClassificationInterpretation.from_learner(learn)
  mostconf.append(interp.confusion_matrix())


Validation Accuracy 10: 0.8372092843055725
Test Accuracy 10: 0.8339645862579346


In [47]:
print(f'Average Validation Accuracy: {calc_mean(valid_avg)}')
print(f'Validation Error: {calcError(np.array(valid_avg))}')
print(f'Average Test Accuracy: {calc_mean(test_avg)}')
print(f'Test Error: {calcError(np.array(test_avg))}')

Average Validation Accuracy: 0.8339403212070465
Validation Error: 0.0012811394288844689
Average Test Accuracy: 0.8373528480529785
Test Error: 0.0005863463749612452
